In [12]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from xlrd import open_workbook
import re
import os
import time
from os import listdir
from os.path import isfile, join
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_datapackage

In [13]:
#out_dir = '../../'
out_dir = '../'

# data to hold all the values
variants = []
#['CONSTANT-FERTILITY', 'ZERO-MIGRATION', 'NOTES', 'HIGH VARIANT', 'ESTIMATES', 'MEDIUM VARIANT', 'NO CHANGE', 'CONSTANT-MORTALITY', 'INSTANT-REPLACEMENT', 'LOW VARIANT']
agegroups = []
#['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80+', '9', '100+', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '0-4', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '5-9', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90-94', '95-99', '0-1', '0-14', '0-17', '0-19', '0-24', '11-16', '11-17', '11-18', '12-14', '12-16', '12-17', '12-18', '13-17', '13-18', '13-19', '15+', '15-17', '15-24', '15-49', '15-59', '15-64', '16+', '17+', '18+', '18-23', '20+', '20-64', '20-69', '21+', '25+', '25-49', '25-64', '25-69', '3-4', '3-5', '3-6', '4-5', '4-6', '5-10', '5-11', '5-14', '50+', '6-10', '6-11', '6-12', '6-9', '60+', '65+', '7-10', '7-12', '70+', '75+', 'Total', '85+', '90+']
ref_AreaCode = []

In [14]:
#method to read files in a folder...
#onlyfiles = [f for f in listdir('source/byYearInterval') if isfile(join('source/byYearInterval', f))]
#onlyfiles

In [15]:
def createDirectory(Directory):
    if not os.path.exists('../'+Directory.lower()):
        os.makedirs('../'+Directory.lower())

In [16]:
def load_Files(source, variant, gender, TypeBy):
    #load file using pandas
    data = pd.read_excel(source, sheetname= variant, skiprows=16, na_values='…')
    data = data.drop(['Index', 'Notes'], axis = 1)
    
    #remove the empty space and convert '-' to '_' char
    data['Variant'] = data['Variant'].str.lower().replace(' ', '').replace('-','')
    
    #rename country column and country code column
    data = data.rename(columns={
        'Major area, region, country or area *': 'Ref_Area',
        'Country code': 'Ref_Area_Code'
    })
    #year column is present in Age Type sheets
    if (TypeBy == "Age"):
        data = data.rename(columns={
        'Reference date (as of 1 July)': 'Year'
    })
    #insert Gender column
    data.insert(3, 'Gender', gender)
    if (TypeBy == "YearInterval"):
        data.insert(3, 'Freq', '5Yearly')
    elif (TypeBy == "AgeYearInterval"):
        data = data.rename(columns={
        'Period': 'Year'
        })
        #get first year in yyyy-yyyy time period
        data['Year'] = data['Year'].str[:4] 
        data.insert(3, 'Freq', '5Yearly')

    #update teh AreaCode entity list
    global ref_AreaCode
    ref_AreaCode = ref_AreaCode + list(set(data['Ref_Area_Code']) - set(ref_AreaCode))

    return data
    

In [17]:
#generate files from the data points.
def GenerateYearFormatFiles(ds_all, FileNameWithPath, gender):
    if (gender.upper() == 'TOTAL') or (gender.upper() == "NA"):
        ds_all = ds_all.drop('gender', axis=1)
    for geo, idxs in ds_all.groupby(by='ref_area_code').groups.items():
        #if geo is world then create world file
        #if geo is continent then create continent file
        #if geo is region then create region file
        #if geo is country then create country file
        path = os.path.join(out_dir,FileNameWithPath.format(geo))
        to_save = ds_all.ix[idxs]
        to_save.ix[idxs].to_csv(path, index=False, float_format='%.15g')

In [18]:
def GetDataFromWorkBookSheets(source, gender, indicator, TypeBy):
    all_variants = []

    wbb = open_workbook(filename = source)
    #add variant concepts
    global variants
    variants = variants + list(set(wbb.sheet_names()) - set(variants))
    
    #iterate through each SHEET except "NOTES"
    for sheetName in wbb.sheet_names():
        if(sheetName == 'NOTES'):
            #ignore NOTES sheet since we are not collecting metadata yet
            continue
        else:
            #first load the files
            mydata = load_Files(source, sheetName, gender, TypeBy)
            mydata = mydata.drop(['Ref_Area'], axis=1)
            #based on File format type apply the index and set the column value
            if (TypeBy == "Age"):
                mydata = mydata.set_index(['Ref_Area_Code','Year','Variant','Gender'])
                mydata.columns.name = 'Age'
            elif (TypeBy == "Year"):
                mydata = mydata.set_index(['Ref_Area_Code','Variant', 'Gender'])
                mydata.columns.name = 'Year'
            elif (TypeBy == "YearInterval"):
                mydata = mydata.set_index(['Ref_Area_Code','Variant', 'Gender', 'Freq'])
                mydata.columns.name = 'Year'
            elif (TypeBy == "AgeYearInterval"):
                mydata = mydata.set_index(['Ref_Area_Code','Year','Variant','Gender', 'Freq'])
                mydata.columns.name = 'Age'
            
            mydata = mydata.stack().reset_index().rename(columns={0:indicator})
            all_variants.append(mydata)
            
            #break
    return all_variants

In [19]:
#method to sort the file with refArea, Year, Variant, Gender
def sortDataSets(dsSet_all, TypeBy):
    dataSet = pd.concat(dsSet_all, ignore_index=True) 
    dataSet.columns = list(map(to_concept_id, dataSet.columns))
    global agegroups
    if (TypeBy == "Age"): 
        #update the global age groups with any new values
        agegroups = agegroups + list(set(dataSet['age'].unique()) - set(agegroups))

        dataSet['age'] = dataSet['age'].astype('category', categories=list(dataSet['age'].unique()), ordered=True)
        dataSet = dataSet.sort_values(by=['ref_area_code', 'year','age','variant', 'gender'])
    elif (TypeBy == "Year"):
        dataSet = dataSet.sort_values(by=['ref_area_code', 'year','variant', 'gender'])
    elif (TypeBy == "YearInterval"):
        dataSet['year'] = dataSet['year'].str[:4]
        dataSet = dataSet.sort_values(by=['ref_area_code', 'year','variant', 'gender', 'freq'])
    elif (TypeBy == "AgeYearInterval"):
        agegroups = agegroups + list(set(dataSet['age'].unique()) - set(agegroups))

        dataSet['age'] = dataSet['age'].astype('category', categories=list(dataSet['age'].unique()), ordered=True)
        dataSet = dataSet.sort_values(by=['ref_area_code', 'year','age','variant', 'gender', 'freq'])
   
    return dataSet

In [20]:
def generateConcepts(metadata):
    # add all the concepts
    concept_name = ['Year', 'Age', 'Gender', 'Freq','Variant' , 'Ref_Area_Code']
    
    concept_name = concept_name + list(metadata.Indicator.unique())
    #print(concept_name)
    
    concepts = list(map(to_concept_id, concept_name))
    cdf = pd.DataFrame([], columns=['concept', 'concept_type'])
    cdf['concept'] = concepts
    cdf['concept_type'] = 'measure'
    cdf['concept_type'].iloc[1:6] = 'entity_domain'
    cdf['concept_type'].iloc[0] = 'time'
    
    path = os.path.join(out_dir, 'ddf--concepts.csv')
    cdf.to_csv(path, index=False)

def generateEntities_Gender():
    cdf = pd.DataFrame([], columns=['gender'])
    cdf['gender'] = ['male', 'female']
    
    path = os.path.join(out_dir, 'ddf--entities--gender.csv')
    cdf.to_csv(path, index=False)

def generateEntities_Freq():
    cdf = pd.DataFrame([], columns=['freq'])
    cdf['freq'] = ['5yearly']
    
    path = os.path.join(out_dir, 'ddf--entities--freq.csv')
    cdf.to_csv(path, index=False)
def generateEntities_Variant():
    variant = [x.lower().replace(' ', '').replace('-', '') for x in variants if x != 'NOTES']
    
    cdf = pd.DataFrame([], columns=['variant'])
    cdf['variant'] = variant
    
    path = os.path.join(out_dir, 'ddf--entities--variant.csv')
    cdf.to_csv(path, index=False)
def generateEntities_AgeGroups():
    cdf = pd.DataFrame([], columns=['age'])
    cdf['age'] = [ x.replace('-','_').replace('+','plus').replace('Total','total').strip() for x in sorted(set(agegroups))]
    
    path = os.path.join(out_dir, 'ddf--entities--age.csv')
    cdf.to_csv(path, index=False)
def generateEntities_RefAreaCode():
    cdf = pd.DataFrame([], columns=['ref_area_code'])
    cdf['ref_area_code'] = ref_AreaCode
    
    path = os.path.join(out_dir, 'ddf--entities--ref_area_code.csv')
    cdf.to_csv(path, index=False)
    

In [21]:
#MAIN Function. Calls the metadata.xslx file and iterate through each file
#supports reading multiple files and concatenating them to one dataset as well.
def callDataPointFiles(metadata_df):
    MainStart = time.time()
    for i, row in enumerate(metadata_df.values):
        start = time.time()
        
        #date = metadata_df.index[i]
        FileName, TypeBY, SEX, Indicator, Directory = row
        newFileName = Directory.lower()
        newFileName = newFileName.replace("ref_area_code", "ref_area_code-{}")
        newFileName = newFileName + ".csv"

        #if (TypeBY == "Age" and Indicator.lower() == "population"):
        if (TypeBY == "Year" or TypeBY == "Age" or TypeBY == "YearInterval" or TypeBY == "AgeYearInterval"):
            print( str(i+1) + ' of '+ str(metadata_df.shape[0]) +' -- ' + FileName)
            #check if FileName has two or more files. for ex same dimensions with male and female
            if(";" in FileName):
                #if yes then run the method for each files and merge the two dataset.
                files = FileName.split(";")
                ds_allSex = []
                #iterate through files then
                for file in files:
                    if "_MALE" in file:
                        SEX = "male"
                    elif "_FEMALE" in file:
                        SEX = "female"
                    ds_sex = GetDataFromWorkBookSheets("source/"+file, SEX, Indicator, TypeBY) 
                    ds_allSex.append(ds_sex)
                    
                 #   break
                #concat all the sheets and files together as one list
                mainds = []
                for dss in ds_allSex:
                    for dss1 in dss:
                        mainds.append(dss1)
                dataSet = sortDataSets(mainds, TypeBY)
            else:
                ds_sex = GetDataFromWorkBookSheets("source/"+FileName, SEX, Indicator, TypeBY) 
                dataSet = sortDataSets(ds_sex, TypeBY)

            #put 3 decimal data for age format. Since some files have data for 80+ as 3.4439999 and 3.444 which
            #is leading to the duplicate error at validate-ddf
            if (TypeBY == "Age"): 
                dataSet.loc[dataSet['age'] == "80+", Indicator.lower()] = np.round(dataSet[Indicator.lower()],decimals=3)
                
            dataSet = dataSet.drop_duplicates()
    

            #create files if df is not empty
            if not dataSet.empty:
                createDirectory(Directory.lower())
                GenerateYearFormatFiles(dataSet, Directory.lower()+'/'+newFileName, SEX)
                #GenerateYearFormatFiles(dataSet, newFileName, SEX)
            else:
                print ('File with no data?? Something WRONG?: ' + FileName )
            
            end = time.time()
            
            print ('Total Time Taken: ' + str(end-start))
            
    print ('Total Time For All Files: ' + str(end-MainStart))
        
        
            #break

In [11]:
metadata_df = pd.read_excel('source/metadata.xlsx', parse_cols = "A:E")

callDataPointFiles(metadata_df)

#generate concepts
generateConcepts(metadata_df)

#generate variants
generateEntities_Variant()

#generate countries/world/regions
generateEntities_RefAreaCode()

#generate age groups
generateEntities_AgeGroups()

#generate gender
generateEntities_Gender()

#generate freq
generateEntities_Freq()

#print (concepts)

# add year interval information for freqency/
# split ref_area_code into World/Continent/Region/Country


1 of 74 -- WPP2015_POP_F01_1_TOTAL_POPULATION_BOTH_SEXES.XLS
Total Time Taken: 7.022952079772949
2 of 74 -- WPP2015_POP_F01_2_TOTAL_POPULATION_MALE.XLS;WPP2015_POP_F01_3_TOTAL_POPULATION_FEMALE.XLS
Total Time Taken: 11.61220097541809
3 of 74 -- WPP2015_POP_F04_SEX_RATIO_OF_TOTAL_POPULATION.XLS
Total Time Taken: 2.424436092376709
4 of 74 -- WPP2015_POP_F05_MEDIAN_AGE.XLS
Total Time Taken: 2.367860794067383
5 of 74 -- WPP2015_POP_F06_POPULATION_DENSITY.XLS
Total Time Taken: 6.696600914001465
6 of 74 -- WPP2015_POP_F11_A_TOTAL_DEPENDENCY_RATIO_1564.XLS
Total Time Taken: 2.2004971504211426
7 of 74 -- WPP2015_POP_F11_B_TOTAL_DEPENDENCY_RATIO_2064.XLS
Total Time Taken: 2.295866012573242
8 of 74 -- WPP2015_POP_F11_C_TOTAL_DEPENDENCY_RATIO_2069.XLS
Total Time Taken: 2.1936159133911133
9 of 74 -- WPP2015_POP_F11_D_TOTAL_DEPENDENCY_RATIO_2564.XLS
Total Time Taken: 2.2517309188842773
10 of 74 -- WPP2015_POP_F11_E_TOTAL_DEPENDENCY_RATIO_2569.XLS
Total Time Taken: 2.195479154586792
11 of 74 -- WPP2